# 北海道 報道発表資料

北海道が発表した新型コロナの患者情報のスクレイピングを行います。詳細はpdfファイルでスクレイピングできないため、患者一覧だけです。

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import bs4
import re
import csv
import datetime
import requests
from collections import defaultdict

In [3]:
#export
urls = ['http://www.pref.hokkaido.lg.jp/hf/kth/kak/hasseijoukyou.htm']

In [4]:
#export
def trim_div(s):
    return re.sub('<div>|<div .+?>|</div>', '', s)

In [5]:
#export
def get_src(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    s = trim_div(response.text)
    return bs4.BeautifulSoup(s, 'html.parser')

In [6]:
#export
def get_links(urls):
    domain = 'http://www.pref.hokkaido.lg.jp'
    filter = '新型コロナウイルス.+患者.+発生'
    links = []
    for url in urls:
        src = get_src(url)
        links.extend(src.find_all(lambda tag: tag.name == 'a' and re.search(filter, tag.get_text()) != None))
    return [[domain + tag['href'], tag.get_text()] for tag in links]

In [7]:
links = get_links(urls)
links

[['http://www.pref.hokkaido.lg.jp/file.jsp?id=1289117',
  '新型コロナウイルスに関連した患者（２６６～２６７例目）の発生について（報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1289114',
  '新型コロナウイルスに関連した新たな患者（２５６～２６５例目）の発生について（札幌市報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1289113',
  '新型コロナウイルスに関連した新たな患者（２４４～２５５例目）の発生について（札幌市報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1289112',
  '新型コロナウイルスに関連した患者（２４０～２４３例目）の発生について（報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1289043',
  '新型コロナウイルスに関連した新たな患者（２３２～２３９例目）の発生について（札幌市報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1289018',
  '新型コロナウイルスに関連した患者（２２７～２３１例目）の発生について（報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1288690',
  '新型コロナウイルスに関連した新たな患者（２２６例目）の発生について（旭川市報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1288682',
  '新型コロナウイルスに関連した新たな患者（２１８～２２５例目）の発生について（札幌市報道発表資料:PDF)'],
 ['http://www.pref.hokkaido.lg.jp/file.jsp?id=1288608',
  '新型コロナウイルスに関連した患者（２０９～２１７例目）の発生について（報道発表資料:P

In [8]:
# export
def parse_links(links):
    refs = {}
    for link in links:
        nums = [re.sub("\\D", "", s) for s in re.split('[～、]', link[1])]
        if len(nums) == 1:
            refs[str(int(nums[0]))] = link[0]
        else:
            for i in range(int(nums[0]), int(nums[1])+ 1):
                refs[str(i)] = link[0]
    return refs

In [9]:
refs = parse_links(links)
#refs

In [10]:
src = get_src(urls[0])
#src

In [11]:
def show_contents(url):
    src = get_src(url)
    
    def get_text(tag):
        return tag.get_text().strip()
    
    return src.body.find(lambda tag: tag.name == 'tbody')

In [12]:
src = show_contents(urls[0])
#src

In [13]:
#export
header = ['No','公表日','年代','性別','居住地','周囲の患者の発生','濃厚接触者の状況','url']

In [14]:
#export
def parse(src, url, refs):
    patients = []
    tbody = src.body.find(lambda tag: tag.name == 'tbody')
    for tr in tbody.find_all('tr'):
        row = []
        for td in tr.find_all('td'):
            text = []
            for c in [c.string for c in td.contents if c.string]: # <br/> を削除
                if c != '\xa0' and c != '\n' and c != '\r\n' and len(c.strip()) != 0: # <br/> で区切られていたこれらの文字を削除
                    text.append(str(re.sub('[\xa0\r\n]', '', c))) # 文字列の一部として含まれる '\xa0' を削除
            row.append('\n'.join(text))
        if row[0].isdecimal() : row[0] = str(int(row[0])) # No を ascii へ変換
        row.append(refs.get(row[0], url)) # url を追加
        patients.append(row)
    return patients[1:]

In [15]:
#export
def get_patient(url):
    links = get_links(url)
    refs = parse_links(links)
    src = get_src(url[0])
    return parse(src, url[0], refs)

In [16]:
patients = get_patient(urls)
#patients

In [17]:
def get_sample(urls, i):
    links = get_links(urls)
    return get_src(links[i][0])
#get_sample(urls, 0)

In [18]:
#export
def create_fname(base):
    now = datetime.datetime.now()
    return base + '_' + now.strftime('%Y%m%dT%H%M') + ".csv"

In [19]:
#export
def write_csv(patients, fname):
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(patients)

In [20]:
#export
def main():
    patients = sorted(get_patient(urls), key=lambda x: int(x[0]))
    patients.insert(0, header)
    fname = create_fname("data/01-pref-hokkaido")
    write_csv(patients, fname)
    print("created:", fname)

In [21]:
#export
if __name__ == '__main__':
    main()

created: data/01-pref-hokkaido_20200412T1854.csv


In [22]:
# See: https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py
!python notebook2script.py 01-pref-hokkaido_dev.ipynb

Converted 01-pref-hokkaido_dev.ipynb to exp/nb_01-pref-hokkaido.py
